In [ ]:
import sys
sys.path.append("../../../")
import csv
import matplotlib.pyplot as plt
import numpy as np
import focusadd
from focusadd.surface.Surface import Surface
from focusadd.coils.CoilSet import CoilSet
from focusadd.lossFunctions.DefaultLoss import default_loss
import numpy as np
import mayavi as maya
from mayavi import mlab
from functools import partial
mlab.init_notebook('x3d',800,800)

In [ ]:
surface = Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0)
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, l_fil = CoilSet.get_outputs(coil_data_fil, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, params_fil)

In [ ]:
def draw_coils(r_coils, color = "blue"):
    if color.lower() == "blue":
        tup = (0.0, 0.0, 0.8)
    elif color.lower() == "red":
        tup = (0.8, 0.0, 0.0)
    elif color.lower() == "pink":
        tup = (1.0, 0.75, 0.8)
    else:
        tup = (0.0, 0.8, 0.0)
    for ic in range(r_coils.shape[0]):
        for n in range(r_coils.shape[2]):
            for b in range(r_coils.shape[3]):
                p = mlab.plot3d(r_coils[ic,:,n,b,0], r_coils[ic,:,n,b,1], r_coils[ic,:,n,b,2], tube_radius = 0.004, line_width = 0.01, color = tup)
    return p

def draw_centroid(r_centroid, color="blue"):
    if color.lower() == "blue":
        tup = (0.0, 0.0, 0.8)
    elif color.lower() == "red":
        tup = (0.8, 0.0, 0.0)
    elif color.lower() == "pink":
        tup = (1.0, 0.75, 0.8)
    else:
        tup = (0.0, 0.8, 0.0)
    for ic in range(r_centroid.shape[0]):
        p = mlab.plot3d(r_centroid[ic,:,0], r_centroid[ic,:,1], r_centroid[ic,:,2], tube_radius = 0.004, line_width = 0.01, color = tup)
    return p

def draw_surface(surface):
    r = surface.get_r()
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
    return p

mlab.clf()
draw_surface(Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0))
draw_coils(r_fil)

In [ ]:
coil_data_fb, params_fb = CoilSet.get_initial_data(surface, input_file="fb.hdf5")
_, _, r_fb, l_fb = CoilSet.get_outputs(coil_data_fb, params_fb)
r_centroid_fb = CoilSet.get_r_centroid(coil_data_fb, params_fb)

In [ ]:
mlab.clf()
draw_surface(Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0))
draw_coils(r_fb)

In [ ]:
coil_data_rot, params_rot = CoilSet.get_initial_data(surface, input_file="fb_rot.hdf5")
_, _, r_rot, l_rot = CoilSet.get_outputs(coil_data_rot, params_rot)
r_centroid_rot = CoilSet.get_r_centroid(coil_data_rot, params_rot)

In [ ]:
mlab.clf()
draw_surface(Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0))
draw_coils(r_rot)

In [ ]:
coil_data_sk, params_sk = CoilSet.get_initial_data(surface, input_file="skinnycoils.hdf5")
_, _, r_sk, l_sk = CoilSet.get_outputs(coil_data_sk, params_sk)
r_centroid_sk = CoilSet.get_r_centroid(coil_data_sk, params_sk)

In [ ]:
mlab.clf()
draw_surface(Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0))
draw_coils(r_sk)

In [ ]:
mlab.clf()
draw_surface(Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0))
draw_coils(r_rot, color="blue")
draw_coils(r_fb, color="green")

In [ ]:
mlab.clf()
draw_centroid(r_centroid_fil, color="red")
draw_centroid(r_centroid_fb, color="blue")
draw_centroid(r_centroid_rot, color="green")
draw_centroid(r_centroid_sk, color="pink")

In [ ]:
def compute_f_B(NZ, NT, NS):
    surface = Surface("../../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", NZ, NT, 1.0)
    w_L = 0.1
    w_B = 1e3
    w_args = (w_B, w_L)
    coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
    NC, _, NF, NFR, ln, lb, NNR, NBR, rc, NR = coil_data_fil
    coil_data_fil = NC, NS, NF, NFR, ln, lb, NNR, NBR, rc, NR
    _, _, r_fil, l_fil = CoilSet.get_outputs(coil_data_fil, params_fil)
    r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, params_fil)
    surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
    coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil)
    loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
    f_B = loss_fil - w_L * l_fil
    return f_B

In [ ]:
NT = 16
NS = 64
NZs = [16, 32, 64, 96, 128, 192, 256, 512, 1024, 2048]
f_Bs_NZ = []
for NZ in NZs:
    f_B = compute_f_B(NZ, NT, NS)
    f_Bs_NZ.append(f_B)

In [ ]:
f_B = compute_f_B(4096, NT, NS)
f_Bs_NZ.append(f_B)
NZs.append(4096)

In [ ]:
print(float((f_Bs_NZ[-1] - f_Bs_NZ[4]) / f_Bs_NZ[4]) * 100)
plt.plot(NZs, f_Bs_NZ)
#plt.ylim([0.0884,0.0887])
plt.title("Quadratic flux vs num_zeta on surface")
plt.show()

In [ ]:
NTs = [8, 16, 32, 64, 128, 256, 512]
NS = 64
NZ = 128
f_Bs_NT = []
for NT in NTs:
    f_B = compute_f_B(NZ, NT, NS)
    f_Bs_NT.append(f_B)

In [ ]:
print(float((f_Bs_NT[-1] - f_Bs_NT[1]) / f_Bs_NT[1]) * 100)
plt.plot(NTs, f_Bs_NT)
#plt.ylim([0.08856,0.08857])
plt.title("Quadratic flux vs num_theta on surface")
plt.show()

In [ ]:
NT = 16
NSs = [16, 32, 64, 96, 128, 256, 512]
NZ = 128
f_Bs_NS = []
for NS in NSs:
    f_B = compute_f_B(NZ, NT, NS)
    f_Bs_NS.append(f_B)

In [ ]:
float((f_Bs_NS[-1] - f_Bs_NS[2]) / f_Bs_NS[2]) * 100

In [ ]:
plt.plot(NSs, f_Bs_NS)
#plt.ylim([0.08775,0.0888])
plt.title("Quadratic flux vs num_segments in coils")
plt.show()

In [ ]:
def plot_surface(r):
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
    return p
mlab.clf()
p = plot_surface(surface.get_r_central()[0:40,:])
#r_c = compute_r_centroid(NS, fc_init)
#p = plot_coils(r_fil, color=(0.8,0.0,0.0))
p = draw_coils(r_fb[2:4])
p = draw_coils(r_rot[4:6],color="green")

In [ ]:
mlab.view(azimuth=270, elevation=90, distance=2.7, focalpoint=(0.45,0.7,0.0), roll=None, reset_roll=True)
p

In [ ]:
print(r_rot[4])